In [1]:
import sys
sys.path.append('../')

import trimesh
import pyvista as pv
from pyvista import PolyData, Plotter
import numpy as np
from irregular_object_packing.mesh.transform import scale_and_center_mesh, scale_to_volume
from irregular_object_packing.mesh.utils import print_mesh_info
from irregular_object_packing.packing import initialize, nlc_optimisation as nlc, chordal_axis_transform as cat
from irregular_object_packing.packing.growth_based_optimisation import SimSettings, Optimizer
from irregular_object_packing.packing.plots import generate_tinted_colors
from importlib import reload

In [2]:
DATA_FOLDER = "../data/mesh/"

mesh_volume = 0.5
container_volume = 10

loaded_mesh = pv.read(DATA_FOLDER + "RBC_normal.stl")
container = pv.Sphere(radius=1)

# Scale the mesh and container to the desired volume
container = scale_to_volume(container, container_volume)
original_mesh = scale_and_center_mesh(loaded_mesh, mesh_volume)

settings = SimSettings(
    itn_max=1,
    n_scaling_steps=1,
    final_scale=0.3,
    sample_rate=100,
    r=0.3
    # plot_intermediate=True,
)
optimizer = Optimizer(original_mesh, container, settings)
optimizer.setup()


PolyData (0x16bba6320)
  N Cells:    200
  N Points:   102
  N Strips:   0
  X Bounds:   -1.347e+00, 1.347e+00
  Y Bounds:   -1.336e+00, 1.336e+00
  Z Bounds:   -1.354e+00, 1.354e+00
  N Arrays:   0

PolyData (0x16bba6320)
  N Cells:    100
  N Points:   52
  N Strips:   0
  X Bounds:   -7.467e-01, 7.417e-01
  Y Bounds:   -2.110e-01, 2.085e-01
  Z Bounds:   -7.464e-01, 7.482e-01
  N Arrays:   0



## Optimisation Algorithm

The next cell will execute the packing algorithm by calling `optimizer.run()`. This will perform some initial setup based on the settings, it will generate the initial positions and rotations of the objects and it will start the optimisation algorithm. depending on the settings, this can take some time.

The optimisation works as follows:

First, the scaling barrier values are determined. These are the limits to which scale the objects can grow in each step (Described in 4.2.2 and Algorithm 1 in the paper). The scaling_barrier values are determined by the `n_scaling_steps` and the `final_scale` parameter. The default value is 0.1, which means that the objects can grow by 10% in each scaling step.

Then for each scaling step, the following itteration is performed `itn_max` times. In each iteration, the following steps are performed:

_The code can be found here: [irregular-object-packing/packing/growth_based_optimisation.py](../irregular_object_packing/packing/growth_based_optimisation.py)_

### step 1: Compute the CAT cells

```Python
        # TRANSFORM MESHES TO OBJECT COORDINATES, SCALE, ROTATION
        obj_points = [
            trimesh.transform_points(self.shape.points.copy(), nlc.construct_transform_matrix(transform_data))
            for transform_data in self.tf_arrs
        ]

        # COMPUTE CAT CELLS
        self.cat_data = cat.compute_cat_cells(obj_points, self.container.points, self.object_coords)
```

### Step 2: Compute the new optimal transform

```Python
        for obj_id, transform_data_i in enumerate(self.tf_arrs):
        tf_arr = optimal_local_transform(
            obj_id,
            self.cat_data,
            scale_bound=(self.settings.init_f, None),
            max_angle=self.settings.max_a,
            max_t=self.settings.max_t,
            margin=self.margin,
        )

        new_tf = transform_data_i + tf_arr
        new_scale = new_tf[0]
        if new_scale > max_scale:
            new_tf[0] = max_scale
```

The `local_optimisation` function is the core of the optimisation algorithm. It performs the following steps:

1. compute the new optimal transform using the non-linear constraint optimisation based on a local coordinate system
2. update the transformation based on the new optimal transform.
3. setting the new object coordinates.


In [3]:
optimizer.run()


scaling 	:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration	:   0%|          | 0/1 [00:00<?, ?it/s]

Object	:   0%|          | 0/7 [00:00<?, ?it/s]

/opt/homebrew/anaconda3/envs/irop/lib/python3.10/site-packages/pyvista/core/dataset.py:2668: PyVistaDeprecationWarning: `cell_points` is deprecated. Use `cell[i].points` instead
  warnings.warn(


## Visualisation

The following cell shows a visualisation of the final result.


In [4]:
plotter = Plotter()
tints = generate_tinted_colors(optimizer.n_objs)

for i, mesh in enumerate(optimizer.final_meshes_after(optimizer.shape)):
    plotter.add_mesh(mesh, color=tints[1][i], opacity=0.8)

for i, mesh in enumerate(optimizer.final_cat_meshes()):
    plotter.add_mesh(mesh, color=tints[0][i], opacity=0.5)

plotter.add_mesh(optimizer.container, color="grey", opacity=0.3)
# plotter.add_mesh(optimizer.container, color="grey", opacity=0.2)

plotter.show(
    interactive=False,
)


Widget(value="<iframe src='http://localhost:65346/index.html?ui=P_0x16be33a90_0&reconnect=auto' style='width: …

In [5]:
import irregular_object_packing.packing.plots as plots
from importlib import reload
reload(plots)
object_id = 1
iteration = 0
second = 1

# cat_cell_mesh_1 = cat.cat_mesh_from_data(optimizer.cat_log[second], object_id)


plots.plot_step_comparison(
    optimizer.mesh_before(iteration, object_id, optimizer.shape),
    optimizer.mesh_after(iteration, object_id, optimizer.shape),
    optimizer.cat_mesh(iteration, object_id),
)


Widget(value="<iframe src='http://localhost:65346/index.html?ui=P_0x16ceb1120_2&reconnect=auto' style='width: …

In [7]:
# Plotting the final step before and after optimization
plots.plot_full_comparison(
    optimizer.final_meshes_before(optimizer.shape),
    optimizer.final_meshes_after(optimizer.shape),
    optimizer.final_cat_meshes(),
    optimizer.container,
).show()


Widget(value="<iframe src='http://localhost:65346/index.html?ui=P_0x16bc34bb0_4&reconnect=auto' style='width: …